In [1]:

import pandas as pd 
import json
import numpy as np
import networkx as nx
import scipy
from scipy import sparse
from scipy.sparse import csc_matrix
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from networkx import to_numpy_matrix, degree_centrality, betweenness_centrality, shortest_path_length,read_edgelist, set_node_attributes
from sklearn.metrics import average_precision_score,recall_score,precision_score,accuracy_score
import os
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math
import time
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from utlis import get_spectrum_embedding,load_data
from sklearn import preprocessing
from scipy.sparse.linalg import inv,eigs
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from scipy.sparse import csc_matrix
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score,recall_score,precision_score,accuracy_score
from plot_utlis import scatter_plot
from train_utlis import train,evaluate, loss,accuracy,get_train_test_set,fit_predict_lr,get_tsne_results
from models import *
from tqdm import tqdm,tqdm_notebook
from utlis import *

# T-SNE

In [1]:
def get_tsne_results(features,n_components=2):
    tsne = TSNE(n_components=n_components, verbose=0, perplexity= 25, n_iter=500)
    tsne_results=tsne.fit_transform(features)
    return tsne_results

In [ ]:



data_set='pubmed'
data,A_norm, features, label_lists, _,_,_=load_data(data_set)
abel_lists=label_lists.numpy()
#data ,features, label_lists=load_data(data_set)


time_start = time.time()

tsne_results=get_tsne_results(features)
print("Training time",time.time()-time_start)
    

In [4]:
x_train, x_test, y_train, y_test =get_train_test_set(tsne_results, label_lists, train_test_split_coef=0.8)

In [5]:
# train test split

fit_predict_lr(x_train, x_test, y_train, y_test)

Train acc:  0.5939897292842199
Training precision:  0.5924447294932744 Training recall:  0.5939897292842199
Test acc:  0.5816430020283976
Testing precision:  0.5791011575811594 Training recall:  0.5816430020283976


(0.5939897292842199, 0.5816430020283976)

# Spectrum Embedding

In [8]:

data_set='pubmed'

data,A_norm, features, label_lists, _,_,_=load_data(data_set)
label_lists=label_lists.numpy()
k=31
time_start = time.time()

sp_results=get_spectrum_embedding(data,k)

print("Training time",time.time()-time_start)
x_train, x_test, y_train, y_test =get_train_test_set(sp_results, label_lists, train_test_split_coef=0.8)

fit_predict_lr(x_train, x_test, y_train, y_test)

Training time 1.1480820178985596
Train acc:  0.7879287389843403
Training precision:  0.7876430907366762 Training recall:  0.7879287389843403
Test acc:  0.7849898580121704
Testing precision:  0.787814941403923 Training recall:  0.7849898580121704


(0.7879287389843403, 0.7849898580121704)

In [ ]:
tsne_results=get_tsne_results(sp_results)

In [ ]:
plot=scatter_plot(tsne_results,label_lists,data_set=data_set)

In [11]:

# data_set='github_user'
# data, features, label_lists=load_data(data_set)

# k=2

# time_start = time.time()

# sp_results=get_spectrum_embedding(data,k)

# print("Training time",time.time()-time_start)
# x_train, x_test, y_train, y_test =get_train_test_set(sp_results, label_lists, train_test_split_coef=0.8)
# fit_predict_lr(x_train, x_test, y_train, y_test)

In [ ]:
plot=scatter_plot(sp_results,label_lists,data_set=data_set)

# Graph Convolution Network

In [10]:
DATASET=['cora','citeseer','pubmed']
dataset_str='cora'
_,A_norm, features, labels, idx_train,idx_test,idx_val=load_data(dataset_str)

no_cuda= False,
seed=42
epochs=400
lr=0.01
weight_decay=5e-4
hidden=16
dropout=0.5
fast_training=False
device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
early_stop_epoch=10

    

In [11]:

# dataset_str='github_user'
# data,features, labels=load_data(dataset_str)


# A=get_adjacency_matrix(data)

# A_norm=preprocess_adj(A)


# no_cuda= False,
# seed=42
# epochs=400
# lr=0.02
# weight_decay=0
# hidden=32
# dropout=0
# fast_training=False
# device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
# early_stop_epoch=10

    

In [12]:
full_idx=np.arange(len(features)).reshape(-1,1)
labels=torch.LongTensor(labels)
features=torch.FloatTensor(features)
label_lists=labels.numpy()
x_train, x_test, y_train, y_test =get_train_test_set(full_idx, label_lists, train_test_split_coef=0.8)

idx_train = torch.LongTensor(x_train.reshape(-1))
idx_test = torch.LongTensor(x_test.reshape(-1))


model = SimpleGCN(adj=A_norm,nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout,res_connection=True,get_hidden=True).to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)


In [8]:


# Train model
t_total = time.time()
epoch_no_improvement=0
min_val_loss=float('inf')

# init_loss_train, init_acc_train,_=evaluate(model,features,idx_train,labels)
# training_loss_list=[init_loss_train]
# training_error_list=[1-init_acc_train]
       
# if not fast_training:
#     init_loss_val, init_acc_val,_=evaluate(model,features,idx_val,labels)
#     validation_loss_list=[init_loss_val]
#     validation_error_list=[1-init_acc_val]

for epoch in tqdm_notebook(range(epochs)):
    
    loss_train,acc_train,output,hidden=train(model,features,idx_train,labels,optimizer)
    
    #print('Epoch: {:04d} ,loss_train: {:.4f},acc_train: {:.4f}\n'.format(epoch+1,loss_train,acc_train),end="")
          
    if not fast_training:
        
        loss_val,acc_val,output,hidden=evaluate(model,features,idx_test,labels)
  
       #print('Eval: ,loss_val: {:.4f},acc_val: {:.4f}\n'.format(loss_val,acc_val),end="")
        if loss_val<min_val_loss:
          epoch_no_improvement=0
          min_val_loss = loss_val
        else:
          epoch_no_improvement+=1
        
        if epoch_no_improvement==early_stop_epoch:
          print('Early Stop at Epoch %d'%(epoch))
          break
        #print()
          
print("Optimization Finished!")
print("Total Training time elapsed: {:.4f}s".format(time.time() - t_total))


#test
print("Start Inference:")


loss_trn,acc_trn,_,hidden=evaluate(model,features,idx_train,labels)
print('Train:','loss;: {:.4f}'.format( loss_trn),
          'acc: {:.4f}'.format(acc_trn))


loss_test,acc_test,_,_=evaluate(model,features,idx_test,labels)
print('Test:','loss;: {:.4f}'.format( loss_test),
          'acc: {:.4f}'.format(acc_test))


Early Stop at Epoch 202
Optimization Finished!
Total Training time elapsed: 5.8745s
Start Inference:
Train: loss;: 0.2536 acc: 0.9626
Test: loss;: 0.4850 acc: 0.8679


In [18]:
tsne_results=get_tsne_results(hidden.detach().numpy())

[t-SNE] Computing 76 nearest neighbors...
[t-SNE] Indexed 19717 samples in 0.026s...
[t-SNE] Computed neighbors for 19717 samples in 1.095s...
[t-SNE] Computed conditional probabilities for sample 1000 / 19717
[t-SNE] Computed conditional probabilities for sample 2000 / 19717
[t-SNE] Computed conditional probabilities for sample 3000 / 19717
[t-SNE] Computed conditional probabilities for sample 4000 / 19717
[t-SNE] Computed conditional probabilities for sample 5000 / 19717
[t-SNE] Computed conditional probabilities for sample 6000 / 19717
[t-SNE] Computed conditional probabilities for sample 7000 / 19717
[t-SNE] Computed conditional probabilities for sample 8000 / 19717
[t-SNE] Computed conditional probabilities for sample 9000 / 19717
[t-SNE] Computed conditional probabilities for sample 10000 / 19717
[t-SNE] Computed conditional probabilities for sample 11000 / 19717
[t-SNE] Computed conditional probabilities for sample 12000 / 19717
[t-SNE] Computed conditional probabilities for sam

In [ ]:
plot=scatter_plot(tsne_results,label_lists,data_set=dataset_str)